In [1]:
import pandas as pd
import numpy as np
import string

import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import tensorflow.keras.utils as ku

tf.random.set_seed(2021)

In [2]:
df = pd.read_csv('/content/buzzfeed_headlines.csv')
df.head()

,content,description,title
0,CBS Films\r\nI couldn't find the second video ...,Warning: This is gonna get pretty cute.View En...,16 Wholesome Details From TV And Movies That S...
1,"HBO Max\r\n""When I was working with Alexa last...","""I needed to have something different in my ey...",Jacob Elordi Has Been Living The Dream Filming...
2,"""Honestly working from home HAS been a game ch...","""Work will never love you back.""View Entire Po...",People Are Sharing Things That Helped Them Get...
3,Mike Marsland / WireImage via Getty Images\r\n...,You really can't win in Hollywood.View Entire ...,"7 Actors Who Were Considered Too ""Ugly"" For A ..."
4,Jane Barlow - Pa Images / PA Images via Getty ...,"""It would have been a money spinner.""",Brian Cox Says He Turned Down A Role In “Pirat...


In [3]:
df.drop(['content', 'description'], axis=1, inplace=True)

In [4]:
title_text = df['title'].tolist()
print(title_text[:5])

['16 Wholesome Details From TV And Movies That Show They Were Quite Literally Made With Love', 'Jacob Elordi Has Been Living The Dream Filming Passionate Love Scenes With Sydney Sweeney On "Euphoria," And He Knows It', "People Are Sharing Things That Helped Them Get A Better Work-Life Balance, And It's So Refreshing", '7 Actors Who Were Considered Too "Ugly" For A Role, And 8 Who Weren\'t Ugly Enough', 'Brian Cox Says He Turned Down A Role In “Pirates Of The Caribbean”']


In [5]:
def clean_text(text):
  text = ''.join(w for w in text if w not in string.punctuation).lower()
  text = text.encode('utf8').decode('ascii', 'ignore')

  return text

corpus = [clean_text(w) for w in title_text]

In [6]:
print(corpus[:15])

['16 wholesome details from tv and movies that show they were quite literally made with love', 'jacob elordi has been living the dream filming passionate love scenes with sydney sweeney on euphoria and he knows it', 'people are sharing things that helped them get a better worklife balance and its so refreshing', '7 actors who were considered too ugly for a role and 8 who werent ugly enough', 'brian cox says he turned down a role in pirates of the caribbean', 'eat at this dessert buffet like a toddler at a birthday party and ill tell you which bsc character you remind me of', 'directors must really love these 17 pairs of actors together since they keep getting cast as costars', 'all of the major pop culture happenings this week from celebrity tributes to new trailers', 'aussies are sharing how their experiences overseas have changed their view on australia and the responses are fascinating', 'telling you not to discuss your pay with coworkers and 18 other red flags in the workplace', 'i

In [7]:
tokenizer = Tokenizer()
def get_sequences(corpus):
  # create tokens
  tokenizer.fit_on_texts(corpus)
  total_words = len(tokenizer.word_index) + 1

  # convert to sequences
  input_sequences = []
  for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
      n_gram_sequence = token_list[:i+1]
      input_sequences.append(n_gram_sequence)

  return input_sequences, total_words

inp_sequences, total_words = get_sequences(corpus)
inp_sequences[:10]

[[130, 413],
 [130, 413, 204],
 [130, 413, 204, 18],
 [130, 413, 204, 18, 63],
 [130, 413, 204, 18, 63, 2],
 [130, 413, 204, 18, 63, 2, 97],
 [130, 413, 204, 18, 63, 2, 97, 8],
 [130, 413, 204, 18, 63, 2, 97, 8, 118],
 [130, 413, 204, 18, 63, 2, 97, 8, 118, 23],
 [130, 413, 204, 18, 63, 2, 97, 8, 118, 23, 44]]

In [8]:
# tokenizer = Tokenizer()

# def get_sequence_of_tokens(corpus):
#   # tokenization
#   tokenizer.fit_on_texts(corpus)
#   total_words = len(tokenizer.word_index) + 1

#   # convert data to sequence of tokens
#   input_sequences = []
#   for line in corpus:
#     token_list = tokenizer.texts_to_sequences([line])[0]
#     for i in range(1, len(token_list)):
#       n_gram_sequence = token_list[:i+1]
#       input_sequences.append(n_gram_sequence)
#   return input_sequences, total_words

# inp_sequences, total_words = get_sequence_of_tokens(corpus)
# inp_sequences[:10]

In [9]:
# padding the sequences
def generate_padded_sequences(input_sequences):
  max_sequence_len = max([len(x) for x in input_sequences])
  input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
  predictors, label = input_sequences[:,:-1], input_sequences[:, -1]
  label = ku.to_categorical(label, num_classes=total_words)
  return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [10]:
def create_model(max_sequence_len, total_words):
  input_len = max_sequence_len - 1
  model = Sequential()

  # Input Embedding Layer
  model.add(Embedding(total_words, 50, input_length=input_len))

  # Add a Hidden Layer - LSTM Layer
  model.add(LSTM(128, return_sequences=True))
  model.add(Dropout(0.2))

  # Second Hidden Layer
  model.add(LSTM(128))
  model.add(Dropout(0.2))

  # Add Output Layer
  model.add(Dense(total_words, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam')
  return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 50)            174800    
                                                                 
 lstm (LSTM)                 (None, 32, 128)           91648     
                                                                 
 dropout (Dropout)           (None, 32, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 3496)              450984    
                                                                 
Total params: 849,016
Trainable params: 849,016
Non-trai

In [11]:
history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
433/433 [==============================] - 8s 9ms/step - loss: 7.1431
Epoch 2/100
433/433 [==============================] - 4s 9ms/step - loss: 6.7363
Epoch 3/100
433/433 [==============================] - 4s 9ms/step - loss: 6.6174
Epoch 4/100
433/433 [==============================] - 4s 9ms/step - loss: 6.5225
Epoch 5/100
433/433 [==============================] - 4s 9ms/step - loss: 6.4060
Epoch 6/100
433/433 [==============================] - 4s 9ms/step - loss: 6.2827
Epoch 7/100
433/433 [==============================] - 4s 9ms/step - loss: 6.1581
Epoch 8/100
433/433 [==============================] - 4s 9ms/step - loss: 6.0347
Epoch 9/100
433/433 [==============================] - 4s 9ms/step - loss: 5.9197
Epoch 10/100
433/433 [==============================] - 5s 10ms/step - loss: 5.8054
Epoch 11/100
433/433 [==============================] - 4s 10ms/step - loss: 5.6986
Epoch 12/100
433/433 [==============================] - 4s 9ms/step - loss: 5.5938
Epoch 13/10

In [12]:

def generate_text(seed_text, next_words, model, max_sequence_len):
  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list, verbose=0))

    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += " " + output_word

  return seed_text.title()

In [13]:
print(generate_text("taylor swift didn't", 12, model, max_sequence_len))
print(generate_text("all the things you wish", 15, model, max_sequence_len))
print(generate_text("the best actors that", 20, model, max_sequence_len))
print(generate_text("things you wish you had", 10, model, max_sequence_len))
print(generate_text("celebrities you never knew", 8, model, max_sequence_len))

Taylor Swift Didn'T Revealed That Her Grandmother Tippy Hedrens Career Was Ruined Because There In
All The Things You Wish Three Squirrel For Halloween 2021 And Its Actually Super Concerning About His Skinny Music Awards
The Best Actors That Her Week From Find Out Which Things From Both Beyond Reigning Fire Pickup Really Really More History At All Costs
Things You Wish You Had About Sebastian Stans Talking Penis In Pam Tommy Army Heres
Celebrities You Never Knew To Buy To Be The Mess From Thank
